# Ensemble Learning

## Initial Imports

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [6]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [7]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [8]:
# Create our features
y = df["loan_status"]

# Create our target
x_cols = [i for i in df.columns if i not in ('Class','Unnamed: 0')]
X = df.drop(columns="loan_status")

In [9]:
X

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,n,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,n,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,n,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,n,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,n,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,n,9.60,0.0,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,n,29.07,0.0,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,n,14.86,0.0,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,n,9.96,0.0,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder=LabelEncoder()
label_encoder.fit(X["home_ownership"])
X["homeowner_le"] = label_encoder.transform(X["home_ownership"])
X = X.drop(columns="home_ownership")

label_encoder.fit(X["verification_status"])
X["homeowner_le"] = label_encoder.transform(X["verification_status"])
X = X.drop(columns="verification_status")

label_encoder.fit(X["issue_d"])
X["homeowner_le"] = label_encoder.transform(X["issue_d"])
X = X.drop(columns="issue_d")

label_encoder.fit(X["pymnt_plan"])
X["homeowner_le"] = label_encoder.transform(X["pymnt_plan"])
X = X.drop(columns="pymnt_plan")

label_encoder.fit(X["initial_list_status"])
X["initial_list_status_le"] = label_encoder.transform(X["initial_list_status"])
X = X.drop(columns="initial_list_status")

label_encoder.fit(X["next_pymnt_d"])
X["next_pymnt_d_le"] = label_encoder.transform(X["next_pymnt_d"])
X = X.drop(columns="next_pymnt_d")

label_encoder.fit(X["application_type"])
X["application_type_le"] = label_encoder.transform(X["application_type"])
X = X.drop(columns="application_type")

label_encoder.fit(X["hardship_flag"])
X["hardship_flag_le"] = label_encoder.transform(X["hardship_flag"])
X = X.drop(columns="hardship_flag")

label_encoder.fit(X["debt_settlement_flag"])
X["debt_settlement_flag_le"] = label_encoder.transform(X["debt_settlement_flag"])
X = X.drop(columns="debt_settlement_flag")


In [12]:
X

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,homeowner_le,initial_list_status_le,next_pymnt_d_le,application_type_le,hardship_flag_le,debt_settlement_flag_le
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,65687.0,38199.0,2000.0,61987.0,0,1,1,0,0,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,271427.0,60641.0,41200.0,49197.0,0,1,1,0,0,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,60644.0,45684.0,7500.0,43144.0,0,1,1,0,0,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,99506.0,68784.0,19700.0,76506.0,0,1,1,0,0,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,219750.0,25919.0,27600.0,20000.0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,20625.0,6798.0,11300.0,5425.0,0,1,1,0,0,0
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,87939.0,60350.0,13500.0,62939.0,0,1,1,0,0,0
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,30592.0,18611.0,3600.0,18492.0,0,1,1,0,0,0
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,1033574.0,95958.0,100800.0,78634.0,0,0,1,0,0,0


In [13]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [14]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [15]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [16]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [17]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [18]:
# Calculated the balanced accuracy score
y_pred_rf = brf.predict(X_test_scaled)
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred_rf)

0.6825993344932644

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_rf)

array([[   40,    39],
       [ 2417, 14709]], dtype=int64)

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.51      0.86      0.03      0.66      0.42        79
   low_risk       1.00      0.86      0.51      0.92      0.66      0.45     17126

avg / total       0.99      0.86      0.51      0.92      0.66      0.45     17205



In [21]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_

sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.06827497455773644, 'last_pymnt_amnt'),
 (0.067117384420678, 'total_pymnt_inv'),
 (0.06149262074374513, 'total_rec_int'),
 (0.05767938752968682, 'total_rec_prncp'),
 (0.05694062600017372, 'total_pymnt'),
 (0.027965524482876033, 'int_rate'),
 (0.021539014852611033, 'dti'),
 (0.01943494143641954, 'mths_since_recent_inq'),
 (0.018549588513404155, 'annual_inc'),
 (0.018102229732178506, 'max_bal_bc'),
 (0.016838478838440526, 'avg_cur_bal'),
 (0.01638963124031303, 'bc_util'),
 (0.016326840281223807, 'bc_open_to_buy'),
 (0.015740948087922677, 'tot_cur_bal'),
 (0.0154964784893643, 'il_util'),
 (0.014612468152223286, 'installment'),
 (0.014607265838723644, 'mo_sin_old_rev_tl_op'),
 (0.014459755609467217, 'all_util'),
 (0.01441184576106188, 'mo_sin_old_il_acct'),
 (0.014366410796855274, 'total_il_high_credit_limit'),
 (0.014280565406509429, 'out_prncp_inv'),
 (0.014025525948954082, 'tot_hi_cred_lim'),
 (0.013969509809414617, 'total_bc_limit'),
 (0.013601005736934244, 'mths_since_rcnt_il'),
 (

### Easy Ensemble Classifier

In [22]:
# Train the Classifier
X_train, X_test, y_train, y_test = train_test_split(X,y)

from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [27]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test_scaled)
from sklearn.metrics import balanced_accuracy_score
EasyEnsembleClassifier(y_test, y_pred)
balanced_accuracy_score(y_test, y_pred)


0.5160874492806284

In [24]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  49,   32],
       [9808, 7316]], dtype=int64)

In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.00      0.60      0.43      0.01      0.51      0.26        81
   low_risk       1.00      0.43      0.60      0.60      0.51      0.25     17124

avg / total       0.99      0.43      0.60      0.60      0.51      0.25     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Balanced Random Forest has significantly better accuracy score of 68% vrs 51%

2. Which model had the best recall score?

    The Balanced Random Forest has a better recall score (0.86 vs 0.43)

3. Which model had the best geometric mean score?

    The Balanced Random Forest has a better geometric mean score (0.66 vs 0.51)

4. What are the top three features?

    last_pymnt_amnt, total_pymnt_inv, total_rec_int